In [7]:
from datasets import Dataset
import pandas as pd

dataset_path = "nl-logql-dataset"

dataset = Dataset.load_from_disk(dataset_path)
df = dataset.to_pandas()

In [8]:
display(df)

,application,id,question,logql_query,query_explanation,query_result,category
0,openstack,2,How long did it take to spawn instance 3edec1e...,"{application=""openstack"", log_file_type=""nova-...",bla,3edec1e4-9678-4a3a-a21b-a145a4ee5e61 took 20.58,
1,openstack,3,What was the total time taken to build instanc...,"{application=""openstack"", log_file_type=""nova-...","1. {application=""openstack"", log_file_type=""no...",21.38,
2,openstack,4,What was the total time taken to build instanc...,"{application=""openstack"", log_file_type=""nova-...","1\n{application=""openstack"", log_file_type=""no...",vcpu: 0.00 VCPU used out of 16 VCPU\ndisk: 0.0...,
3,openstack,5,What is the vCPU usage for compute node cp-1.s...,max by (node) (\n max_over_time(\n {applic...,"1\n{application=""openstack"", log_file_type=""no...",<graph>\ngraph with plot of used_vcpus across ...,
4,openstack,6,What is the RAM usage for compute node cp-1.sl...,max by (node) (\n max_over_time(\n {applic...,"1\n{application=""openstack"", log_file_type=""no...",<graph>\\ngraph with plot of used_vcpus across...,
...,...,...,...,...,...,...,...
56,openssh,58,Show me all unique IP addresses that attempted...,sum by (ip) (\n count_over_time(\n {applic...,"1\n{application=""openssh"", hostname=""LabSZ""}\n...",226 unique IP addresses,Invalid User Attempts
57,openssh,59,How many attempts were made to authenticate wi...,sum(\nsum by (source_ip) (\n count_over_time(...,"1\n{application=""openssh"", hostname=""LabSZ""}\n...",11.5k,Invalid User Attempts
58,openssh,60,How many times did we encounter 'Corrupted MAC...,sum by (hostname) (count_over_time({applicatio...,"1\n{application=""openssh""}\nFetch all log line...",1,System Health and Performance
59,openssh,61,"List all 'Bad packet length' errors, including...",sum by (length) (\n count_over_time(\n ...,"1\n{application=""openssh"", hostname=""LabSZ""}\n...",1\n<graph>,System Health and Performance


In [4]:
# view all questions where the application is openstack
openstack_df = df[df["application"] == "openstack"]

In [16]:
pd.set_option("display.max_colwidth", None)

In [25]:
print(openstack_df[["question"]].to_json(lines=True, orient="records"))

# # view as json
# openstack_df.to_json("openstack_questions.json", orient="records", lines=True)

{"question":"How long did it take to spawn instance 3edec1e4-9678-4a3a-a21b-a145a4ee5e61 on the hypervisor?"}
{"question":"What was the total time taken to build instance 3edec1e4-9678-4a3a-a21b-a145a4ee5e61?"}
{"question":"What was the total time taken to build instance 3edec1e4-9678-4a3a-a21b-a145a4ee5e61?"}
{"question":"What is the vCPU usage for compute node cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us?"}
{"question":"What is the RAM usage for compute node cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us?"}
{"question":"What was the maximum response time for GET request to \/v2\/54fadb412c4e40cdbaed9335e4c35a9e\/servers\/detail?"}
{"question":"How many POST requests to \/v2\/e9746973ac574c6b8a9e8857f56a7608\/os-server-external-events resulted in a 200 status code?"}
{"question":"What's the average response time for DELETE requests to \/v2\/54fadb412c4e40cdbaed9335e4c35a9e\/servers\/"}
{"question":"How many instances of 'HTTP exception thrown: No instances found for any event' occurred in the n

In [9]:
dataset.push_to_hub("sidbin/natural-logql")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/467 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sidbin/natural-logql/commit/6ed5d768f7c4deedd146614e3a8850b248c5a603', commit_message='Upload dataset', commit_description='', oid='6ed5d768f7c4deedd146614e3a8850b248c5a603', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/sidbin/natural-logql', endpoint='https://huggingface.co', repo_type='dataset', repo_id='sidbin/natural-logql'), pr_revision=None, pr_num=None)

In [29]:
df[df["id"] == 5]["query_result"].values[0]

'<graph>\ngraph with plot of used_vcpus across different logs.\nmaximum is 1\n</graph>'

In [2]:
from datasets import Dataset
import pandas as pd
from IPython.display import display, HTML

dataset_path = "nl-logql-dataset"
# Load existing dataset or create a new one
try:
    dataset = Dataset.load_from_disk(dataset_path)
    df = dataset.to_pandas()
except:
    df = pd.DataFrame(
        columns=[
            "application",
            "id",
            "question",
            "logql_query",
            "query_explanation",
            "query_result",
        ]
    )


def save_dataset():
    dataset = Dataset.from_pandas(df)
    dataset.save_to_disk(dataset_path)


def display_message(message, color="green"):
    display(HTML(f"<p style='color:{color}'>{message}</p>"))


def add_entry(app, question, query, explanation, result):
    global df
    if not all([app, question, query, explanation, result]):
        display_message("All fields must be filled.", "red")
        return
    new_id = df["id"].max() + 1 if len(df) > 0 else 1
    new_row = pd.DataFrame(
        {
            "application": [app],
            "id": [new_id],
            "question": [question],
            "logql_query": [query],
            "query_explanation": [explanation],
            "query_result": [result],
        }
    )
    df = pd.concat([df, new_row], ignore_index=True)
    save_dataset()
    display_message(f"Entry added successfully. Total entries: {len(df)}")


def edit_entry(entry_id, app, question, query, explanation, result):
    global df
    if entry_id not in df["id"].values:
        display_message(f"Entry with ID {entry_id} not found.", "red")
        return
    if not all([app, question, query, explanation, result]):
        display_message("All fields must be filled.", "red")
        return
    df.loc[
        df["id"] == entry_id,
        ["application", "question", "logql_query", "query_explanation", "query_result"],
    ] = [app, question, query, explanation, result]
    save_dataset()
    display_message(f"Entry {entry_id} updated successfully.")


def delete_entry(entry_id):
    global df
    if entry_id not in df["id"].values:
        display_message(f"Entry with ID {entry_id} not found.", "red")
        return
    df = df[df["id"] != entry_id].reset_index(drop=True)
    save_dataset()
    display_message(f"Entry {entry_id} deleted successfully.")


def display_entries():
    display(df)


# Input cells for adding a new entry
def add_new_entry():
    app = input("Application name: ")
    question = input("Question: ")
    query = input("LogQL query: ")
    explanation = input("Query explanation: ")
    result = input("Query result: ")
    add_entry(app, question, query, explanation, result)


# Input cells for editing an existing entry
def edit_existing_entry():
    entry_id = int(input("Enter the ID of the entry to edit: "))
    app = input("Application name: ")
    question = input("Question: ")
    query = input("LogQL query: ")
    explanation = input("Query explanation: ")
    result = input("Query result: ")
    edit_entry(entry_id, app, question, query, explanation, result)


# Input cell for deleting an entry
def delete_existing_entry():
    entry_id = int(input("Enter the ID of the entry to delete: "))
    delete_entry(entry_id)


# Display current dataset
display_entries()

,application,id,question,logql_query,query_explanation,query_result


In [3]:
add_new_entry()